# BioSCape Data Access

The BioSCape Data is being stored in an S3 bucket that is attached to the SMCE environment. This bucket is accessible using the s3 api or any package that uses the s3 api.
The bucket is read only.

To run this example you need to have s3fs and rioxarry installed in a virutal environment.

**Currently the reflectance data is missing their header files. Hopefully this will be corrected soon**

In [1]:
import rioxarray as rxr
import os
import s3fs

In [2]:
s3 = s3fs.S3FileSystem(anon=False)
files = s3.ls('bioscape-data/')
files

['bioscape-data/ang20231025t060824',
 'bioscape-data/ang20231025t063547',
 'bioscape-data/ang20231025t072700',
 'bioscape-data/ang20231025t073616',
 'bioscape-data/ang20231025t074923',
 'bioscape-data/ang20231025t075757',
 'bioscape-data/ang20231025t081917',
 'bioscape-data/ang20231025t082814',
 'bioscape-data/ang20231025t092131',
 'bioscape-data/ang20231025t102340',
 'bioscape-data/ang20231109t063500',
 'bioscape-data/ang20231109t082143',
 'bioscape-data/ang20231109t141759',
 'bioscape-data/ang20231113t111341',
 'bioscape-data/ang20231113t112511',
 'bioscape-data/ang20231113t113448',
 'bioscape-data/ang20231113t115523',
 'bioscape-data/ang20231113t121555',
 'bioscape-data/ang20231113t130615',
 'bioscape-data/ang20231113t131251',
 'bioscape-data/ang_quicklooks']

In [3]:
sub_files = s3.ls('bioscape-data/ang20231025t060824')
sub_files

['bioscape-data/ang20231025t060824/ang20231025t060824_rdn_glt',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_glt.hdr',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_igm',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_igm.hdr',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_loc_ort',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_loc_ort.hdr',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_obs_ort',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_obs_ort.hdr',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_ort',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rdn_ort.hdr',
 'bioscape-data/ang20231025t060824/ang20231025t060824_rfl_ort']

In [10]:
print(f"File Name: {sub_files[8]}")
rxr.open_rasterio(os.path.join('s3://', sub_files[8]))

File Name: bioscape-data/ang20231025t060824/ang20231025t060824_rdn_ort


<xarray.DataArray (band: 425, y: 19204, x: 1107)> Size: 36GB
[9035001900 values with dtype=float32]
Coordinates:
    fwhm         (band) float64 3kB 5.57 5.58 5.58 5.58 ... 6.02 6.02 6.02 6.03
    wavelength   (band) float64 3kB 377.2 382.2 387.2 ... 2.496e+03 2.501e+03
  * band         (band) int64 3kB 1 2 3 4 5 6 7 ... 419 420 421 422 423 424 425
    xc           (y, x) float64 170MB 6.442e+05 6.442e+05 ... 8.608e+05
    yc           (y, x) float64 170MB 6.229e+06 6.229e+06 ... 6.212e+06
    spatial_ref  int64 8B 0
Dimensions without coordinates: y, x
Attributes: (12/443)
    Band_1:              channel_0 (377.1956495 Nanometers)
    Band_10:             channel_9 (422.27564950000004 Nanometers)
    Band_100:            channel_99 (873.0556495000001 Nanometers)
    Band_101:            channel_100 (878.0656495 Nanometers)
    Band_102:            channel_101 (883.0756494999999 Nanometers)
    Band_103:            channel_102 (888.0756494999999 Nanometers)
    ...                  ...
    file_type:           ENVI Standard
    header_offset:       0
    interleave:          bil
    lines:               19204
    masked_pixel_noise:  2.6014206409454346
    samples:             1107